# Cost Analysis of Predicted Futures




## Cost data comparison


Importing necessary packages


In [58]:
import base64
import pandas as pd
import duckdb
import numpy as np
import scipy
import json
from fhir.resources.riskassessment import RiskAssessment
from fhir.resources.reference import Reference
from fhir.resources.codeableconcept import CodeableConcept
from IPython.display import Image, display

Loading data


In [2]:
predicted_futures = pd.read_parquet(
    "predicted_futures_data.parquet"
)
with open("cost_distributions.json") as json_file:
    cost_origin = json.load(json_file)

As we can see, from this patient, the values could be different for one prediction within and across days_until_event


In [59]:
special_case = duckdb.query(
    """
select *
FROM predicted_futures 
where patient_id = 624603 and prediction_id = 8 and lower(key) in ('plan_paid_total_amount','policy_paid_total_amount')
"""
).to_df()
special_case

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,patient_id,prediction_id,key,value,value_numeric,days_until_event
0,624603,8,policy_paid_total_amount,NaN,143.919998,0
1,624603,8,policy_paid_total_amount,NaN,143.919998,0
2,624603,8,policy_paid_total_amount,NaN,143.919998,0
3,624603,8,policy_paid_total_amount,NaN,143.919998,0
4,624603,8,policy_paid_total_amount,NaN,154.070007,200
5,624603,8,policy_paid_total_amount,NaN,125.239998,200


- deduplicate records
- pick keys only in plan_paid_total_amount or policy_paid_total_amount
- compute the average cost for a given patient, prediction


In [60]:
cost_df_event = duckdb.query(
    """

with dedup_all as (
    select distinct *
    from predicted_futures
),

predicted_futures_cleaned as (
select patient_id, prediction_id, key, value_numeric
from dedup_all where lower(key) in ('plan_paid_total_amount','policy_paid_total_amount') 
),

dedup_avg as(
select patient_id, prediction_id, key, avg(value_numeric) as avg_cost
from predicted_futures_cleaned
group by patient_id, prediction_id, key
)
select *
from dedup_avg

"""
).to_df()
cost_df_event

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,patient_id,prediction_id,key,avg_cost
0,34902651,2,policy_paid_total_amount,90.229996
1,35067291,2,policy_paid_total_amount,106.639999
2,35067291,6,policy_paid_total_amount,131.009995
3,35199003,4,policy_paid_total_amount,8.060000
4,35363643,0,policy_paid_total_amount,70.019997
...,...,...,...,...
82566,879439995,2,policy_paid_total_amount,537.220032
82567,879571707,0,policy_paid_total_amount,0.000000
82568,880361979,7,policy_paid_total_amount,0.000000
82569,920567067,14,policy_paid_total_amount,0.000000


In order to generate FHIR resource,
prediction_id and cost are nested for each patient


In [61]:
cost_df_event_agg = duckdb.query(
    """
select patient_id, 
  list(prediction_id) as prediction_id_list, 
  list(avg_cost) as avg_cost_list
from cost_df_event
group by patient_id

"""
).to_df()
cost_df_event_agg

,patient_id,prediction_id_list,avg_cost_list
0,35067291,"[2, 6, 5, 15, 3, 9, 1, 12, 4, 13, 8]","[106.63999938964844, 131.00999450683594, 134.7..."
1,39677211,"[14, 1, 3, 5, 9, 13, 11, 2, 10, 6]","[199.63999938964844, 102.77999877929688, 31.73..."
2,41784603,"[5, 6, 12, 3, 13, 15, 9]","[0.0, 0.30000000447034836, 1.5749999284744263,..."
3,82845819,"[0, 13, 4, 9, 1, 5, 14, 2, 12, 6]","[2.3450001254677773, 0.5, 0.5, 0.0, 2.42000007..."
4,157954587,"[11, 9, 1, 5, 7, 2, 10]","[107.95999908447266, 107.95999908447266, 159.1..."
...,...,...,...
9602,828829659,[4],[162.69000244140625]
9603,569159451,"[13, 8]","[156.69000244140625, 0.0]"
9604,718026939,[13],[0.0]
9605,149722587,[9],[285.5299987792969]


For comparing the distribution of orginal cost and predicted cost, the cost data are further aggregated for each patients


In [62]:
cost_df = duckdb.query(
    """
with final as (
select patient_id, sum(avg_cost) as total_cost
from cost_df_event
group by patient_id)
select *
from final
"""
).to_df()

In [63]:
cost_df_summary = cost_df.total_cost.describe()
cost_df_summary

count      9607.000000
mean       1319.170638
std        6347.825679
min           0.000000
25%         246.051665
50%         666.175001
75%        1285.403496
max      341015.028625
Name: total_cost, dtype: float64

Cost Comparison Metrics

- t test
- standardized mean difference (SMD)

In a addition, if we have the complete real cost data, we can use k-s test for distribution comparison


Since costs are continuous variables, and large sample, t test can be used for comparing the distribution of the mean. From the t test result, we can see that the distribution of the mean is statistically significantly different from each other.


In [64]:
# t test
scipy.stats.ttest_ind_from_stats(
    cost_df_summary.mean(),
    cost_df_summary.std(),
    cost_df.shape[0],
    cost_origin["mean"],
    cost_origin["std"],
    cost_origin["count"],
    equal_var=False,
    alternative="two-sided",
)

Ttest_indResult(statistic=35.16592659880916, pvalue=3.545687212718715e-255)

standardized mean difference were calculated, which is greater than 0.1, also suggest the difference between the two groups.


In [65]:
smd = (cost_df_summary.mean() - cost_origin["mean"]) / np.sqrt(
    (cost_df_summary.std() ** 2 + cost_origin["std"] ** 2) / 2
)
smd

0.5074473663879351

## FHIR resources

steps:

- turn nested dataframe cost_df_event_agg to dict
- generate Reference object for patient id
- prediction elements
  - generate outcome key using CodeableConcept with prediction id
  - generate probabilityDecimal for storing cost prediction. It is possible that there is a better place to store cost.
- finally, generate RiskAssessment resource


In [33]:
cost_dict_df = cost_df_event_agg.to_dict()
list_resource = []
for i in range(cost_df_event_agg.shape[0]):
    pat_ref = Reference()
    pat_ref.reference = f"Patient/{cost_dict_df['patient_id'][i]}"

    prediction_list = [
        {
            "outcome": CodeableConcept(
                text=f"Cost prediction_id {cost_dict_df['prediction_id_list'][i][j]} "
            ),
            "probabilityDecimal": cost_dict_df["avg_cost_list"][i][j],
        }
        for j in range(len(cost_dict_df["prediction_id_list"][i]))
    ]

    obj_ra = RiskAssessment(status="final", subject=pat_ref)
    obj_ra.prediction = prediction_list
    list_resource.append(obj_ra)

In [34]:
list_resource[1]

RiskAssessment(resource_type='RiskAssessment', fhir_comments=None, id=None, implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, contained=None, extension=None, modifierExtension=None, text=None, basedOn=None, basis=None, code=None, condition=None, encounter=None, identifier=None, method=None, mitigation=None, mitigation__ext=None, note=None, occurrenceDateTime=None, occurrenceDateTime__ext=None, occurrencePeriod=None, parent=None, performer=None, prediction=[RiskAssessmentPrediction(resource_type='RiskAssessmentPrediction', fhir_comments=None, extension=None, id=None, modifierExtension=None, outcome=CodeableConcept(resource_type='CodeableConcept', fhir_comments=None, extension=None, id=None, coding=None, text='Cost prediction_id 9 ', text__ext=None), probabilityDecimal=Decimal('117.25999450683594'), probabilityDecimal__ext=None, probabilityRange=None, qualitativeRisk=None, rationale=None, rationale__ext=None, relativeRisk=None, relativeRisk__ext=N

## Pipeline design


In [70]:
def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


mm(
    """
flowchart TB

A(Client data)--> b(QC) --> ELT --> a(Feature Engineering w/ Feature Store) --> c(QC) --> MLOps --> e(FHIR resource) --> f(QC) --> g(Client)

subgraph MLOps
    direction LR
    1(model training) --> 2(model testing) --> 3(model versioning) --> CI/CD --> Inference --> d(QC,data drifting,monitering) --> 1
end
"""
)

Tools as an example

- Orchestration tools: Airflow with Celery and RabbitMQ
- QC: Great Expectation
- ELT: dbt with duckdb
- Feature Store: Feast
- MLOps: MLflow, it is a continuous learning process
- Dataviz for Monitoring, Logging, Metrics: Grafana
- Alerting: Slack/email
- CI/CD: precommit, github action
- documentation: dbt doc, great expectation doc, mkdocs
